## This program inferring UML by LLM4Models   

In [ ]:
# Choose 1 to abstract UML from Java
# Choose 2 to abstract UML from Python

What_I_Want=2

In [ ]:
import time
import torch

from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

In [ ]:
DEVICE="cuda:0" if torch.cuda.is_available() else "cpu"

In [ ]:
mistral_checkpoint = "mistralai/Mistral-7B-v0.3"
model = AutoModelForCausalLM.from_pretrained(mistral_checkpoint, torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(mistral_checkpoint, use_fast=True, add_prefix_space=True)
tokenizer.pad_token = tokenizer.unk_token   
tokenizer.padding_side = "left" 
model.eval() 
model.to(DEVICE)

In [ ]:
if What_I_Want==1:
   CheckPoint='HA-Siala/Java-UML' 
else:      
   CheckPoint='HA-Siala/Python-UML' 
    
peft_model = PeftModel.from_pretrained(model, CheckPoint, torch_dtype=torch.bfloat16, is_trainable=False)   

In [ ]:
def GeneratePrompt(content):
   if What_I_Want==1:
      Instruction="""Generate a concise UML class diagram for the provided Java code. The output should:
1. Define each class and interface only once, including its attributes, methods, and relationships.
2. Include all relationships (Inheritance, Realization, Dependency, Association, Composition, Aggregation) without duplication.
3. Avoid redundant or repeated operations, classes, or relationships."""
   else:
      Instruction="""Generate a concise UML class diagram for the provided Python code. The output should:
1. Define each class and interface only once, including its attributes, methods, and relationships.
2. Include all relationships (Inheritance, Realization, Dependency, Association, Composition, Aggregation) without duplication.
3. Avoid redundant or repeated operations, classes, or relationships."""

   prompt = f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response, which is in JSON format that appropriately solves the following Task:

### Instruction:
{Instruction}

### Input:
{content}

### Response:
"""   
   return prompt

In [ ]:
def GenerateInferenceOutput(text): 
   tokenizer.pad_token = tokenizer.unk_token  
   inputs = tokenizer(GeneratePrompt(text), return_tensors="pt").to(DEVICE)
   inputs_length = len(inputs["input_ids"][0])
   with torch.inference_mode():
      outputs = model.generate(
         **inputs,  
         max_new_tokens= 8000, 
         temperature= 0.2, 
         do_sample= True, 
         pad_token_id=tokenizer.eos_token_id,
         top_p=0.9,
         use_cache=True,
      )
      output_p = tokenizer.batch_decode(outputs, skip_special_tokens=True)
      if output_p:
         output_text = output_p[0]
         split_text_p = output_text.split("Response:")
         return split_text_p[1].strip() if len(split_text_p) > 1 else None
      else:
         return None

In [ ]:
# Put the program here between """ and """
# You need to remove comments and empty lines (pre-processing)
content=""""""

In [ ]:
StartTime=time.time()
Output = GenerateInferenceOutput(content)
EndTime=time.time()
InferenceTime=EndTime - StartTime
print(Output)
print()
print(f"Inference Time: {InferenceTime:.2f} seconds")